<a href="https://colab.research.google.com/github/MushroomFleet/StableCascade-text2image/blob/main/Johnsons_Fork_240217_T4_free_colab_of_Stable_Cascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## T4 Colab - Stable Cascade Txt2img
Original Author @mkshing - Forked by Drift Johnson
tested running with T4 and no high memory option. restricted to 1024x512 max or it will explode

# INSTALL Cascade

In [ ]:
%cd /content/
!git lfs install
!git clone https://huggingface.co/spaces/multimodalart/stable-cascade
%cd /content/stable-cascade
!pip install -r /content/stable-cascade/requirements.txt
!pip3 uninstall torch torchvision torchaudio -y
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip uninstall torchdata torchtext -y
!pip install torchdata torchtext

/content
Git LFS initialized.
fatal: destination path 'stable-cascade' already exists and is not an empty directory.
/content/stable-cascade
  Cloning https://github.com/kashif/diffusers.git (to revision diffusers-yield-callback) to /tmp/pip-req-build-r4a6jpg8
  Running command git clone --filter=blob:none --quiet https://github.com/kashif/diffusers.git /tmp/pip-req-build-r4a6jpg8
  Running command git checkout -b diffusers-yield-callback --track origin/diffusers-yield-callback
  Switched to a new branch 'diffusers-yield-callback'
  Branch 'diffusers-yield-callback' set up to track remote branch 'diffusers-yield-callback' from 'origin'.
  Resolved https://github.com/kashif/diffusers.git to commit c4d20110774dbcc6ada7e62b290500f8ad777fba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://gradio-builds.s3.amazonaws.com/aabb08191a7d94d2a1e9ff87b0d3c3987cd519c5/gradio-4.18.0-py3-none-

# LOAD Cascade

In [ ]:
import torch
import numpy as np
from diffusers import StableCascadeDecoderPipeline, StableCascadePriorPipeline
from diffusers.utils import numpy_to_pil

# Assuming the environment setup and model loading is done as per your script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# Initialize pipelines (ensure this matches your environment's setup)
prior_pipeline = StableCascadePriorPipeline.from_pretrained("stabilityai/stable-cascade-prior", torch_dtype=dtype).to(device)
decoder_pipeline = StableCascadeDecoderPipeline.from_pretrained("stabilityai/stable-cascade", torch_dtype=dtype).to(device)


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/pyt

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/models/lora.py:384: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

# Generate a image  (1024x512  requires 13.6gb)

In [ ]:
import random
import re


Random_seed = random.randint(0, 6969)

def generate_image(prompt, seed=Random_seed, width=1024, height=1024, prior_guidance_scale=7.5, decoder_guidance_scale=0.0, num_inference_steps=50, num_images_per_prompt=1):
    # Set seed for reproducibility
    Random_seed = random.randint(0, 6969)
    generator = torch.Generator(device=device).manual_seed(seed)

    # Generate with Prior
    prior_results = prior_pipeline(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=prior_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is supported
        generator=generator
    )

    # Extract image embeddings from the generator object
    image_embeddings = None
    for result in prior_results:
        if 'image_embeddings' in result:
            image_embeddings = result['image_embeddings']
            break

    if image_embeddings is None:
        raise ValueError("Failed to generate image embeddings from the prior pipeline")

    # Generate with Decoder
    decoder_output = decoder_pipeline(
        image_embeddings=image_embeddings,
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=decoder_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is correctly applied
        generator=generator,
        output_type="pil"
    )

    return decoder_output.images[0]  # Returns the first (and expected only) generated image

# Example usage
if __name__ == "__main__":
    prompt = "A tabby cat riding on a skateboard down streets of new york"
    Random_seed = random.randint(0, 6969)  # Ensure you have imported random and generated Random_seed

    # Truncate the prompt to a reasonable length and replace spaces with underscores
    safe_prompt = re.sub(r'[^\w\s-]', '', prompt)[:50].strip().replace(' ', '_')

    # Generate the image
    image = generate_image(prompt, num_images_per_prompt=1, seed=Random_seed, width=1024, height=512, prior_guidance_scale=7.5, decoder_guidance_scale=0.0, num_inference_steps=50)

    # Format the filename with the sanitized prompt and seed
    filename = f"/content/{safe_prompt}_{Random_seed}.png"

    # Save the image
    image.save(filename)
    print(f"Image saved as {filename}")
    image.show()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved as /content/A_tabby_cat_riding_on_a_skateboard_down_streets_of_6515.png


In [ ]:
# @markdown ### **Prompt**
pos_prompt = "A tabby cat riding on a skateboard down streets of new york" # @param {type:"string"}
# @markdown ### **Width**
Bwidth = 1024 # @param {type:"number"}
# @markdown ### **Height**
Bheight = 1024 # @param {type:"number"}
# @markdown ### **Width**
Cwidth = 1024 # @param {type:"number"}
# @markdown ### **Height**
Cheight = 512 # @param {type:"number"}

# @markdown ### **Prior Guidance Scale**
Uprior_guidance_scale = 7.5 # @param {type:"number"}
# @markdown ### **Decoder Guidance Scale**
Udecoder_guidance_scale = 0.0 # @param {type:"number"}
# @markdown ### **Steps**
Unum_inference_steps = 50 # @param {type:"number"}
# @markdown ### **Batch**
Unum_images_per_prompt = 1 # @param {type:"number"}

import random
import re


Random_seed = random.randint(0, 6969)

def generate_image(prompt, seed=Random_seed, width=Bwidth, height=Bheight, prior_guidance_scale=Uprior_guidance_scale, decoder_guidance_scale=Udecoder_guidance_scale, num_inference_steps=Unum_inference_steps, num_images_per_prompt=Unum_images_per_prompt):
    # Set seed for reproducibility
    Random_seed = random.randint(0, 6969)
    generator = torch.Generator(device=device).manual_seed(seed)

    # Generate with Prior
    prior_results = prior_pipeline(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=prior_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is supported
        generator=generator
    )

    # Extract image embeddings from the generator object
    image_embeddings = None
    for result in prior_results:
        if 'image_embeddings' in result:
            image_embeddings = result['image_embeddings']
            break

    if image_embeddings is None:
        raise ValueError("Failed to generate image embeddings from the prior pipeline")

    # Generate with Decoder
    decoder_output = decoder_pipeline(
        image_embeddings=image_embeddings,
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=decoder_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is correctly applied
        generator=generator,
        output_type="pil"
    )

    return decoder_output.images[0]  # Returns the first (and expected only) generated image

# Example usage
if __name__ == "__main__":
    prompt = pos_prompt
    Random_seed = random.randint(0, 6969)  # Ensure you have imported random and generated Random_seed

    # Truncate the prompt to a reasonable length and replace spaces with underscores
    safe_prompt = re.sub(r'[^\w\s-]', '', prompt)[:50].strip().replace(' ', '_')

    # Generate the image
    image = generate_image(prompt, num_images_per_prompt=Unum_images_per_prompt, seed=Random_seed, width=Cwidth, height=Cheight, prior_guidance_scale=Uprior_guidance_scale, decoder_guidance_scale=Udecoder_guidance_scale, num_inference_steps=Unum_inference_steps)

    # Format the filename with the sanitized prompt and seed
    filename = f"/content/{safe_prompt}_{Random_seed}.png"

    # Save the image
    image.save(filename)
    print(f"Image saved as {filename}")
    image.show()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved as /content/A_tabby_cat_riding_on_a_skateboard_down_streets_of_5601.png
